# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,axim,4.8699,-2.2405,24.38,95,100,3.11,GH,1700957788
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,-5.67,69,40,11.62,RU,1700957789
2,2,punta arenas,-53.1500,-70.9167,9.06,66,20,16.46,CL,1700957790
3,3,ca mau,9.1769,105.1500,23.96,90,100,0.98,VN,1700957791
4,4,waitangi,-43.9535,-176.5597,16.01,96,100,6.42,NZ,1700957792


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
vacation_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.7,
    color = "City",
    alpha = 0.6
)

# Display the map
vacation_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,axim,4.8699,-2.2405,24.38,95,100,3.11,GH,1700957788
1,1,petropavlovsk-kamchatsky,53.0452,158.6483,-5.67,69,40,11.62,RU,1700957789
2,2,punta arenas,-53.1500,-70.9167,9.06,66,20,16.46,CL,1700957790
3,3,ca mau,9.1769,105.1500,23.96,90,100,0.98,VN,1700957791
4,4,waitangi,-43.9535,-176.5597,16.01,96,100,6.42,NZ,1700957792


In [5]:
# Narrow down cities that fit criteria and drop any results with null values

vacation_city_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) &( city_data_df["Max Temp"] < 27)
                                   & (city_data_df["Wind Speed"]<4.5) & (city_data_df["Cloudiness"] == 0)]

    

# Drop any rows with null values

vacation_city_df = vacation_city_df.dropna()

# Display sample data
vacation_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
69,69,santiago,-33.4569,-70.6483,23.81,57,0,2.07,CL,1700957700
76,76,piton saint-leu,-21.2196,55.3151,21.04,68,0,1.54,RE,1700957853
95,95,ojinaga,29.5667,-104.4167,22.53,33,0,0.00,MX,1700957869
230,230,san luis de la loma,17.2706,-100.8939,26.10,77,0,0.80,MX,1700957991
263,263,tsiombe,-25.3000,45.4833,24.02,68,0,2.73,MG,1700958023
303,303,yanbu,24.0891,38.0637,25.02,83,0,2.06,SA,1700958057
392,392,aioun,16.6614,-9.6149,23.11,15,0,4.06,MR,1700958144
436,436,medina,24.4686,39.6142,22.24,64,0,2.06,SA,1700958183
439,439,lazaro cardenas,17.9583,-102.2000,26.46,73,0,2.04,MX,1700958185
485,485,saint-pierre,-21.3393,55.4781,21.82,68,0,1.54,RE,1700958228


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_city_df.copy(deep=True)
hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "
hotel_df= hotel_df.reset_index(drop=True)

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,santiago,CL,-33.4569,-70.6483,57,
1,piton saint-leu,RE,-21.2196,55.3151,68,
2,ojinaga,MX,29.5667,-104.4167,33,
3,san luis de la loma,MX,17.2706,-100.8939,77,
4,tsiombe,MG,-25.3000,45.4833,68,
5,yanbu,SA,24.0891,38.0637,83,
6,aioun,MR,16.6614,-9.6149,15,
7,medina,SA,24.4686,39.6142,64,
8,lazaro cardenas,MX,17.9583,-102.2000,73,
9,saint-pierre,RE,-21.3393,55.4781,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    "categories":categories,
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lon = row["Lng"]
    lat = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santiago - nearest hotel: apart arturo prat
piton saint-leu - nearest hotel: Appartement Piton Saint Leu
ojinaga - nearest hotel: Hotel
san luis de la loma - nearest hotel: No hotel found
tsiombe - nearest hotel: No hotel found
yanbu - nearest hotel: No hotel found
aioun - nearest hotel: فندق العيون
medina - nearest hotel: The Oberoi Madina
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
saint-pierre - nearest hotel: Tropic Hotel
conde - nearest hotel: Paladino


,City,Country,Lat,Lng,Humidity,Hotel Name
0,santiago,CL,-33.4569,-70.6483,57,apart arturo prat
1,piton saint-leu,RE,-21.2196,55.3151,68,Appartement Piton Saint Leu
2,ojinaga,MX,29.5667,-104.4167,33,Hotel
3,san luis de la loma,MX,17.2706,-100.8939,77,No hotel found
4,tsiombe,MG,-25.3000,45.4833,68,No hotel found
5,yanbu,SA,24.0891,38.0637,83,No hotel found
6,aioun,MR,16.6614,-9.6149,15,فندق العيون
7,medina,SA,24.4686,39.6142,64,The Oberoi Madina
8,lazaro cardenas,MX,17.9583,-102.2000,73,Hotel Sol del Pacífico
9,saint-pierre,RE,-21.3393,55.4781,68,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
#     "Country",
#     "Hotel Name",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    scale = 0.7,
    color = "City",
    alpha = 0.6,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)